In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
import yaml

from pyspark.sql import SparkSession

while not Path("data") in Path(".").iterdir():
    os.chdir("..")

plt.style.use("seaborn-white")
conf_dict = yaml.safe_load(Path("config/dataprep.yaml").read_text())

spark = SparkSession.builder.getOrCreate()

22/05/30 19:16:41 WARN Utils: Your hostname, domvwt-XPS-13-9305 resolves to a loopback address: 127.0.1.1; using 192.168.0.24 instead (on interface wlp164s0)
22/05/30 19:16:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/30 19:16:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/30 19:16:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/30 19:16:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
def repartition(sdf):
    return (
        sdf
        .withColumn("partition", F.substring("statementID", -2, 2))
        .repartition("partition")   
    )

In [3]:
companies_df = repartition(spark.read.parquet(conf_dict["companies_interim"])).select(conf_dict["companies_cols"])
persons_df = repartition(spark.read.parquet(conf_dict["persons_interim"])).select(conf_dict["persons_cols"])
addresses_df = spark.read.parquet(conf_dict["addresses_interim"]).repartition(100)

In [140]:
filter_id = lambda id_ : f"statementID == '{id_}'"

## Relationships

In [138]:
relationships_df = repartition(
    spark.read.parquet(conf_dict["relationships_interim"])
).select(conf_dict["relationships_cols"])

relationships_df = (
    relationships_df.withColumn(
        "interestedPartyStatementID",
        F.coalesce(
            "interestedParty.describedByEntityStatement",
            "interestedParty.describedByPersonStatement",
        ),
    )
    .withColumn(
        "interestedPartyIsPerson",
        F.col("interestedParty.describedByPersonStatement").isNotNull(),
    )
    .withColumn("subjectStatementID", F.col("subject.describedByEntityStatement"))
    .select("*", F.explode("interests").alias("interestsExploded"))
    .select("*", "interestsExploded.*")
    .select("*", F.col("share.minimum").alias("minimumShare"))
    .filter(F.col("startDate").isNotNull())
    .filter(F.col("endDate").isNull())
    .filter(F.col("type") == "shareholding")
    .filter(F.col("details").endswith("percent"))
    .groupBy(
        "interestedPartyStatementID",
        "interestedPartyIsPerson",
        "subjectStatementID",
    )
    .agg(F.max("minimumShare").alias("minimumShare"))
)


In [139]:
relationships_df.show()


+--------------------------+-----------------------+--------------------+------------+
|interestedPartyStatementID|interestedPartyIsPerson|  subjectStatementID|minimumShare|
+--------------------------+-----------------------+--------------------+------------+
|      openownership-reg...|                   true|openownership-reg...|        25.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        25.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        25.0|
|      openownership-reg...|               

In [127]:
query_id = "openownership-register-8389244008015946645"

In [128]:
persons_df.filter(filter_id(query_id)).take(1)

[Row(addresses=[Row(address='Peter House, Oxford Street, Manchester, M1 5AN', country=None, type=None)], birthDate='1960-11-01', identifiers=[Row(id='/company/09073660/persons-with-significant-control/individual/miJk0_Iz1MxSspVDSLOdsIea9z8', scheme=None, schemeName='GB Persons Of Significant Control Register', uri=None), Row(id='http://register.openownership.org/entities/59b9cda867e4ebf340ccfc74', scheme=None, schemeName='OpenOwnership Register', uri='http://register.openownership.org/entities/59b9cda867e4ebf340ccfc74')], missingInfoReason=None, names=[Row(fullName='Donald Hugh Smith', type='individual')], nationalities=[Row(code='GB', name='United Kingdom of Great Britain and Northern Ireland')], statementID='openownership-register-8389244008015946645')]

In [129]:
relationships_df.filter(f"interestedPartyStatementID == '{query_id}'").dropDuplicates().show(truncate=False)

+-------------+------------------------------------------+-----------------------+-------------------------------------------+-------+----------+------------+-----------------+
|statementDate|interestedPartyStatementID                |interestedPartyIsPerson|subjectStatementID                         |endDate|startDate |type        |max(minimumShare)|
+-------------+------------------------------------------+-----------------------+-------------------------------------------+-------+----------+------------+-----------------+
|2016-06-06   |openownership-register-8389244008015946645|true                   |openownership-register-18123958185355667650|null   |2016-06-06|shareholding|50.0             |
+-------------+------------------------------------------+-----------------------+-------------------------------------------+-------+----------+------------+-----------------+



In [130]:
relationships_df.groupBy(F.col("startDate").isNull()).count().orderBy(F.desc("count")).show(truncate=0)

+-------------------+-------+
|(startDate IS NULL)|count  |
+-------------------+-------+
|false              |7416437|
+-------------------+-------+



In [131]:
relationship_counts = relationships_df.groupBy("interestedPartyStatementID", "interestedPartyIsPerson").count().sort(F.desc("count"))
relationship_counts.show(truncate=False)

+-------------------------------------------+-----------------------+-----+
|interestedPartyStatementID                 |interestedPartyIsPerson|count|
+-------------------------------------------+-----------------------+-----+
|openownership-register-15335012948911220969|false                  |1659 |
|openownership-register-5349204506857096345 |true                   |1311 |
|openownership-register-16449901526180129165|true                   |1171 |
|openownership-register-4885699358762453064 |false                  |1131 |
|openownership-register-996612036464718947  |true                   |760  |
|openownership-register-4241702842928979482 |true                   |720  |
|openownership-register-2286052163500911970 |false                  |681  |
|openownership-register-3956438078590756395 |true                   |617  |
|openownership-register-9103080704383038680 |false                  |526  |
|openownership-register-4513814456581281117 |false                  |503  |
|openownersh

In [ ]:
relationship_counts.filter("count == 10 AND interestedPartyIsPerson").show(truncate=False)

+-------------------------------------------+-----------------------+-----+
|interestedPartyStatementID                 |interestedPartyIsPerson|count|
+-------------------------------------------+-----------------------+-----+
|openownership-register-3257107209837998303 |true                   |10   |
|openownership-register-11769433893356591601|true                   |10   |
|openownership-register-6265616255109065414 |true                   |10   |
|openownership-register-15368251881549564308|true                   |10   |
|openownership-register-5299651258949798128 |true                   |10   |
|openownership-register-13469635606482485026|true                   |10   |
|openownership-register-15171453100235327141|true                   |10   |
|openownership-register-17289159870146257517|true                   |10   |
|openownership-register-5902812732140731840 |true                   |10   |
|openownership-register-11764017702979503973|true                   |10   |
|openownersh

In [144]:
target_person_id = "openownership-register-11769433893356591601"
target_person_filter = F.col("interestedPartyStatementID") == target_person_id

In [145]:
relationships_df.filter(target_person_filter).show()

+--------------------------+-----------------------+--------------------+------------+
|interestedPartyStatementID|interestedPartyIsPerson|  subjectStatementID|minimumShare|
+--------------------------+-----------------------+--------------------+------------+
|      openownership-reg...|                   true|openownership-reg...|        25.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
|      openownership-reg...|                   true|openownership-reg...|        75.0|
+--------------------------+-----------------------+--------------------+------------+



## Companies

In [146]:
companies_df.count()

7246475

In [147]:
companies_df.printSchema()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- dissolutionDate: string (nullable = true)
 |-- foundingDate: string (nullable = true)
 |-- identifiers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- scheme: string (nullable = true)
 |    |    |-- schemeName: string (nullable = true)
 |    |    |-- uri: string (nullable = true)
 |-- name: string (nullable = true)
 |-- statementID: string (nullable = true)



In [150]:
(companies_df.select("name", F.explode("identifiers").alias("identifiersExploded"))).select("col.").show()


+--------------------+--------------------+
|                name|                 col|
+--------------------+--------------------+
|PKAN SERVICES LIM...|{10649951, GB-COH...|
|PKAN SERVICES LIM...|{https://opencorp...|
|PKAN SERVICES LIM...|{http://register....|
|      NEUTEK LIMITED|{09460241, GB-COH...|
|      NEUTEK LIMITED|{https://opencorp...|
|      NEUTEK LIMITED|{http://register....|
|THE LEE RESIDENTS...|{03936630, GB-COH...|
|THE LEE RESIDENTS...|{https://opencorp...|
|THE LEE RESIDENTS...|{http://register....|
| J AND A JOINERY LTD|{SC559379, GB-COH...|
| J AND A JOINERY LTD|{https://opencorp...|
| J AND A JOINERY LTD|{http://register....|
|NDIDI ONYEAMA LIM...|{10650588, GB-COH...|
|NDIDI ONYEAMA LIM...|{https://opencorp...|
|NDIDI ONYEAMA LIM...|{http://register....|
|THE POVERTY ALLIANCE|{SC136689, GB-COH...|
|THE POVERTY ALLIANCE|{https://opencorp...|
|THE POVERTY ALLIANCE|{http://register....|
|   TECHIE PLEASE LTD|{10651103, GB-COH...|
|   TECHIE PLEASE LTD|{https://o

In [9]:
companies_df.printSchema()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- birthDate: string (nullable = true)
 |-- dissolutionDate: string (nullable = true)
 |-- entityType: string (nullable = true)
 |-- foundingDate: string (nullable = true)
 |-- identifiers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- scheme: string (nullable = true)
 |    |    |-- schemeName: string (nullable = true)
 |    |    |-- uri: string (nullable = true)
 |-- incorporatedInJurisdiction: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- interestedParty: struct (nullable = true)
 |    |-- describedByEntityStatement: string (nullable = true)
 |    |-- describedByPersonStatement: string (nullable = tru

In [10]:
companies_sample_pdf.dtypes

addresses                     object
birthDate                     object
dissolutionDate               object
entityType                    object
foundingDate                  object
identifiers                   object
incorporatedInJurisdiction    object
interestedParty               object
interests                     object
missingInfoReason             object
name                          object
names                         object
nationalities                 object
personType                    object
source                        object
statementDate                 object
statementID                   object
statementType                 object
subject                       object
partition                     object
dtype: object

In [11]:
companies_sample_pdf.describe().T

,count,unique,top,freq
addresses,9996,9026,"[(20-22 Wenlock Road, London, N1 7GU, GB, regi...",42
birthDate,0,0,NaN,NaN
dissolutionDate,3290,306,2018-01-02,103
entityType,10000,1,registeredEntity,10000
foundingDate,9981,4134,2018-03-26,38
identifiers,10000,10000,"[(10649951, GB-COH, Companies House, None), (h...",1
incorporatedInJurisdiction,10000,1,"(GB, United Kingdom of Great Britain and North...",10000
interestedParty,0,0,NaN,NaN
interests,0,0,NaN,NaN
missingInfoReason,0,0,NaN,NaN


In [12]:
companies_sample_pdf.head().T

,0,1,2,3,4
addresses,"[(Unit 19e Eley Road, Edmonton, London, N18 3B...","[(9 Wellington Gate, East Grinstead, RH19 3GE,...","[(3 The Lee, Northwood, Middlesex, HA6 3HT, GB...","[(First Floor Right 440 George Street, Aberdee...","[(Flat 4 Prudance House, 22 Smithsland Road, R..."
birthDate,None,None,None,None,None
dissolutionDate,None,None,None,2019-04-16,2018-08-07
entityType,registeredEntity,registeredEntity,registeredEntity,registeredEntity,registeredEntity
foundingDate,2017-03-03,2015-02-26,2000-02-24,2017-03-03,2017-03-03
identifiers,"[(10649951, GB-COH, Companies House, None), (h...","[(09460241, GB-COH, Companies House, None), (h...","[(03936630, GB-COH, Companies House, None), (h...","[(SC559379, GB-COH, Companies House, None), (h...","[(10650588, GB-COH, Companies House, None), (h..."
incorporatedInJurisdiction,"(GB, United Kingdom of Great Britain and North...","(GB, United Kingdom of Great Britain and North...","(GB, United Kingdom of Great Britain and North...","(GB, United Kingdom of Great Britain and North...","(GB, United Kingdom of Great Britain and North..."
interestedParty,None,None,None,None,None
interests,None,None,None,None,None
missingInfoReason,None,None,None,None,None


In [13]:
companies_sample_pdf["identifiers"][0]

[Row(id='10649951', scheme='GB-COH', schemeName='Companies House', uri=None),
 Row(id='https://opencorporates.com/companies/gb/10649951', scheme=None, schemeName='OpenCorporates', uri='https://opencorporates.com/companies/gb/10649951'),
 Row(id='http://register.openownership.org/entities/59c5026967e4ebf34060b471', scheme=None, schemeName='OpenOwnership Register', uri='http://register.openownership.org/entities/59c5026967e4ebf34060b471')]

In [14]:
companies_sample_pdf["identifiers"][1]

[Row(id='09460241', scheme='GB-COH', schemeName='Companies House', uri=None),
 Row(id='https://opencorporates.com/companies/gb/09460241', scheme=None, schemeName='OpenCorporates', uri='https://opencorporates.com/companies/gb/09460241'),
 Row(id='http://register.openownership.org/entities/59c5026a67e4ebf34060b67a', scheme=None, schemeName='OpenOwnership Register', uri='http://register.openownership.org/entities/59c5026a67e4ebf34060b67a')]

In [15]:
companies_sample_pdf["incorporatedInJurisdiction"]

0       (GB, United Kingdom of Great Britain and North...
1       (GB, United Kingdom of Great Britain and North...
2       (GB, United Kingdom of Great Britain and North...
3       (GB, United Kingdom of Great Britain and North...
4       (GB, United Kingdom of Great Britain and North...
                              ...                        
9995    (GB, United Kingdom of Great Britain and North...
9996    (GB, United Kingdom of Great Britain and North...
9997    (GB, United Kingdom of Great Britain and North...
9998    (GB, United Kingdom of Great Britain and North...
9999    (GB, United Kingdom of Great Britain and North...
Name: incorporatedInJurisdiction, Length: 10000, dtype: object

One address per entity

In [18]:
companies_sample_pdf["addresses"].map(lambda x: len(x) if x else 0).max()

1

Many identifiers

In [19]:
companies_sample_pdf["identifiers"].map(lambda x: len(x) if x else 0).max()

762

In [20]:
keep_cols = [
    "addresses",
    "dissolutionDate",
    "foundingDate",
    "identifiers",
    # "incorporatedInJurisdiction",
    # "interestedParty",
    # "interests",
    # "missingInfoReason",
    "name",
    # "names",
    # "nationalities",
    # "personType",
    # "source",
    # "statementDate",
    "statementID",
    # "statementType",
    # "subject",
]

In [22]:
companies_df.select(keep_cols).write.parquet(conf_dict["companies_processed"])

22/05/25 22:38:04 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 97.92% for 7 writers
22/05/25 22:38:04 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 85.68% for 8 writers
22/05/25 22:38:05 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 97.92% for 7 writers
22/05/25 22:38:05 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 85.68% for 8 writers
22/05/25 22:38:05 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 97.92% for 7 writers
22/05/25 22:38:05 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 97.92% for 7 writers
22/05/25 22:38:05 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928

## Persons

In [29]:
persons_df.count()

7861347

In [33]:
persons_sample_pdf = persons_df.limit(10_000).toPandas()

In [34]:
persons_sample_pdf.describe().T

,count,unique,top,freq
addresses,10000,9505,"[(20-22, Wenlock Road, London, N1 7GU, GB, None)]",76
birthDate,9987,820,1974-07-01,34
dissolutionDate,0,0,NaN,NaN
entityType,0,0,NaN,NaN
foundingDate,0,0,NaN,NaN
identifiers,10000,9989,[],12
incorporatedInJurisdiction,0,0,NaN,NaN
interestedParty,0,0,NaN,NaN
interests,0,0,NaN,NaN
missingInfoReason,12,5,The company knows or has reasonable cause to b...,6


In [37]:
keep_cols = [
    "addresses",
    "birthDate",
    # "dissolutionDate",
    # "entityType",
    # "foundingDate",
    "identifiers",
    # "incorporatedInJurisdiction",
    # "interestedParty",
    # "interests",
    "missingInfoReason",
    # "name",
    "names",
    "nationalities",
    # "personType",
    # "source",
    # "statementDate",
    "statementID",
    # "statementType",
    # "subject",
    "partition",
]

In [38]:
persons_df.select(keep_cols).write.parquet(conf_dict["persons_processed"])

22/05/25 22:48:46 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 97.92% for 7 writers
22/05/25 22:48:46 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 85.68% for 8 writers
22/05/25 22:48:46 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 97.92% for 7 writers
22/05/25 22:48:46 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 97.92% for 7 writers
22/05/25 22:48:46 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 85.68% for 8 writers
22/05/25 22:48:47 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928 bytes) of heap memory
Scaling row group sizes to 97.92% for 7 writers
22/05/25 22:48:47 WARN MemoryManager: Total allocation exceeds 95.00% (919,941,928